# Preparing the environment and installing libraries:

In [ ]:
!nvidia-smi

In [ ]:
%pip install transformers datasets bitsandbytes peft torch sentencepiece -q
%pip install rouge-score -q
%pip install blobfile tiktoken

In [8]:
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [9]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq
from peft import LoraConfig, get_peft_model

# Model Fine-tuning

### 1. Prepare the data

After the preparation, the dataset looks like this:
```json
{
  "title": "Document title...",
  "content": "Original document text...",
  "summary": "Generated summary..."
}
```
And it's split into `train`, `validation`, and `test` sets.

In [10]:
dataset = load_dataset("json", data_files={"train": "data/"+"train.json", "val": "data/"+"val.json", "test": "data/"+"test.json"})
# print(dataset)

# 2. Custom Quantization

In [11]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

# 3. Model Definition

There are several models interesting to fine-tune for this task. We will use the `transformers` library to fine-tune a model from the list.

In [ ]:
models = {
    "mT5-Base": "google/mt5-base", # 580M params
    "Qwen2.5-0.5B": "Qwen/Qwen2.5-0.5B",
}

model_name = models["Qwen2.5-0.5B"]

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)
# print(f"Loaded model: {model_name}\n", model)
# print(model.config)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
# print(f"Loaded Tokenizer: \n", tokenizer)

Now we will use the tokenizer to encode the input text.

In [ ]:
def prompt_schema(doc):
    return f"""<human>: résume {doc}
<assistant>:"""

def preprocess_function(examples):
    inputs = [ prompt_schema(doc) for doc in examples["content"] ]
    targets = [ doc for doc in examples["summary"] ]
    
    model_inputs = tokenizer(inputs, max_length=8192, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=8192, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return {
        'input_ids': model_inputs["input_ids"],
        'labels': model_inputs["labels"],
        'attention_mask': model_inputs["attention_mask"],
    }

tokenized_datasets = dataset.map(preprocess_function, batched=True)

We will use Lora to fine-tune the model efficiently without having to adjust all the parameters.

In [ ]:
# Extract target_modules
target_modules = [
    'gate_proj',
    'up_proj',
    'down_proj',
]

config = LoraConfig(
    r=32,
    lora_alpha=64,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=target_modules,
)


model = get_peft_model(model, config)
model.print_trainable_parameters()

In [18]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id
generation_config.do_sample = True

In [ ]:
%%time
article = """Le Monument-Lefebvre est un lieu historique national du Canada situé à Memramcook, au Nouveau-Brunswick.\n\n\n== Histoire ==\nLe Monument-Lefebvre est inauguré les 16 et 17 juin 1897 et nommé ainsi en l'honneur du père Camille Lefebvre, fondateur du Collège Saint-Joseph, décédé deux ans auparavant.\nIl est tout d'abord utilisé comme laboratoire par le Collège Saint-Joseph, tandis que son amphithéâtre permet d'organiser des concerts, des conférences et de nombreuses manifestations diverses. Le lieu et son fondateur ont joué un rôle déterminant dans la renaissance acadienne.\nAprès la fermeture du Collège en 1972, le Monument-Lefebvre manque de disparaître mais est finalement rénové et déclaré lieu historique national en 1994.\n\n\n== Architecture ==\nLe Monument-Lefebvre est construit en grès rustiqué de couleur olive. Sa façade symétrique comprend des éléments néoromans. Il abrite un théâtre et des salle de classe.\n\n\n== Notes et références ==\n\n\n== Liens externes ==\nRessources relatives à l'architecture : Édifices fédéraux patrimoniaux Lieux historiques nationaux Répertoire canadien des lieux patrimoniaux \n(fr + en) Site officiel\n\n Portail de l’Acadie   Portail du Nouveau-Brunswick   Portail des lieux patrimoniaux du Canada"""

prompt = f"""
<human>: résume {article}
<assistant>:
"""

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
training_args = TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    num_train_epochs=1,
    learning_rate=2e-4,
    bf16=True,
    save_total_limit=3,
    logging_steps=20,
    output_dir="results",
    max_steps=6,
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
)

In [ ]:
trainer = Trainer(
    model=model,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["val"],
    args=training_args,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model),
)

model.config.use_cache = False
trainer.train()

# 4. Test the fine-tuned model

In [ ]:
%%time

article = """Le Monument-Lefebvre est un lieu historique national du Canada situé à Memramcook, au Nouveau-Brunswick.\n\n\n== Histoire ==\nLe Monument-Lefebvre est inauguré les 16 et 17 juin 1897 et nommé ainsi en l'honneur du père Camille Lefebvre, fondateur du Collège Saint-Joseph, décédé deux ans auparavant.\nIl est tout d'abord utilisé comme laboratoire par le Collège Saint-Joseph, tandis que son amphithéâtre permet d'organiser des concerts, des conférences et de nombreuses manifestations diverses. Le lieu et son fondateur ont joué un rôle déterminant dans la renaissance acadienne.\nAprès la fermeture du Collège en 1972, le Monument-Lefebvre manque de disparaître mais est finalement rénové et déclaré lieu historique national en 1994.\n\n\n== Architecture ==\nLe Monument-Lefebvre est construit en grès rustiqué de couleur olive. Sa façade symétrique comprend des éléments néoromans. Il abrite un théâtre et des salle de classe.\n\n\n== Notes et références ==\n\n\n== Liens externes ==\nRessources relatives à l'architecture : Édifices fédéraux patrimoniaux Lieux historiques nationaux Répertoire canadien des lieux patrimoniaux \n(fr + en) Site officiel\n\n Portail de l’Acadie   Portail du Nouveau-Brunswick   Portail des lieux patrimoniaux du Canada"""

prompt = f"""
<human>: résume {article}
<assistant>:
"""

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

# 3. Save the fine-tuned model

In [ ]:
# Later

# 5. Evaluate the model

### 5.1 ROUGE Score

In [ ]:
from evaluate import load

rouge = load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels)
    return result

results = compute_metrics(trainer.predict(tokenized_datasets["test"]))

print(results)

### 5.2 Bert Score